In [ ]:
# Imports. We only need h80 for the class based approach
from paratc.tc_models import Holland1980 as h80
from paratc import make_example_data
import warnings 
warnings.filterwarnings('ignore')

# Open example track and generate example grid
track, grid_lon, grid_lat = make_example_data()
track

`track` is a pandas dataframe containing tropical cyclone track data. We pass this to `Holland1980()` alongside our 2D grid longitudes and latitudes. The class will then check the track dataframe for needed variables. In the case of `Holland1980()`, the B shape parameter is needed. You can calculate this yourself (using one of the functions in Holland1980), or provide `B_model` to `Holland1980()`, as below.

Our new `storm` object will contain a `data` variable in the form of an xarray dataset. This contains our `grid_lon` and `grid_lat` variables, as well as our initial storm pressure, windspeed and wind components.

In [ ]:
# Create storm instance and look at dataset
storm = h80( track, grid_lon, grid_lat, B_model='vickery00' )
storm.data

We can also check the `storm`'s `track` attribute. This is a copy of the track dataframe, however you will see that it now has a `B` column, translation speeds, `pdelta` and `timestep`:

In [ ]:
storm.track

Let's use `.plot()` to take a look at a snapshot of the storm:

In [ ]:
# Plot storm windspeed at timestep 3
storm.plot(2)

We can see that the windfield is perfectly symmetric, with no inflow towards the storm center. This is because no inflow angle model or background flow model has been applied yet. Let's do that:

In [ ]:
# Scale winds to surface level
storm.scale_winds( 0.91 )

# Apply piecewise inflow angle model
storm.make_wind_vectors( inflow_model = 'wang20' )

# Add background winds using reciprocal relationship, 55% scaling and 20 degree rotation
storm.add_background_winds( bg_alpha = .55, bg_beta = 20 )

There are other options for all models. Take a look into their respective modules for a full list.

In [ ]:
# Plot new storm
storm.plot(2)